In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from keras import backend as K
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os 
import multiprocessing
import wandb
# !pip install wandb -qqq
from wandb.keras import WandbCallback
import kerastuner as kt #!python3.x -m pip install keras-tuner
import cv2
from ipywidgets import fixed, interact 
import ipywidgets
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate, RandomBrightnessContrast, RandomCrop)
from data_utils import Dataset_Classification, Dataset_Segmentation
import sys
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git
sys.path.append('./deeplab')
from deeplabv3p import Deeplabv3
from utils import SegModel, get_VOC2012_classes, Jaccard, sparse_accuracy_ignoring_last_label, sparse_crossentropy_ignoring_last_label

In [ ]:
dataset_config = {
    'train_fraction': 0.9,
    'input_shape': (224, 224),
    'augmentation': True, 
    'uniform_sample_probabilities': False
}
ds = Dataset_Classification(dataset_config)
ds.show_class_distribution()

In [ ]:
stop=0
for (X,y) in ds.train_generator(10):
    stop+=1
    print(X.shape)
    print(y.shape)
    
    if stop>10: 
        break
    

In [ ]:
class RandomClassificationModel:
    """
    Random classification model: 
        - generates random labels for the inputs based on the class distribution observed during training
        - assumes an input can have multiple labels
    """
    def fit(self, X, y):
        """
        Adjusts the class ratio variable to the one observed in y. 

        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
        y: list of arrays - n x (nb_classes)

        Returns
        -------
        self
        """
        self.distribution = np.mean(y, axis=0)
        print("Setting class distribution to:\n{}".format("\n".join(f"{label}: {p}" for label, p in zip(labels, self.distribution))))
        return self
        
    def predict(self, X):
        """
        Predicts for each input a label.
        
        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
            
        Returns
        -------
        y_pred: list of arrays - n x (nb_classes)
        """
        np.random.seed(0)
        return [np.array([int(np.random.rand() < p) for p in self.distribution]) for _ in X]
    
    def __call__(self, X):
        return self.predict(X)
    
    
class ClassifactionModel(RandomClassificationModel): 
    """
        Main class implementing all functions necessary to train and/or use a classification model 
        This class has to be overwritten for each specific model of interest, where the base model should be implemented.
    """
    def __init__(self, config): 
        self.config = config 
        self.config_head = config['head_model']
        
        # initialize dataset
        self.dataset = Dataset_Classification(config['dataset'])
              
        # check if some configurations make sense 
        assert len(self.config_head['head_model_units']) == len(self.config_head['add_dropout']), 'head_models_units and add_dropout list should have same size'
    
    
    def set_config(self, config):
        self.config = config 
        self.config_head = config['head_model']
        self.dataset = Dataset_Classification(config['dataset'])
        assert len(self.config_head['head_model_units']) == len(self.config_head['add_dropout']), 'head_models_units and add_dropout list should have same size'
        
    def predict(self, X):
        # 
        
        if len(X.shape) == 1: 
            # X is a batch of images prepare all of them and create batch. 
            batch = np.array([self.dataset.prepare_test_image(im) for im in X])
            y = model.predict(batch)
        else: 
            # X is a single image 
            batch = self.dataset.prepare_test_image(X)
            batch = np.expand_dims(batch, axis=0)
            y = model.predict(batch)
            
        y = np.squeeze(y)

        label_idx = np.where(y==1.)

        return self.dataset.label_names[label_idx]
            
    def build(self): 
        """
            Builds the model 
        """
#       self.base_model = resnet50
        
        # define a head model
        head_model=keras.layers.GlobalAveragePooling2D()(self.base_model.output)

        for (nbr_units, dropout) in zip(self.config_head['head_model_units'], self.config_head['add_dropout']): 
            head_model=tf.keras.layers.Dense(nbr_units, activation=self.config_head['activation'])(head_model)
            if dropout:
                head_model=tf.keras.layers.Dropout(0.4)(head_model)
        
        head_model=keras.layers.Dense(20, activation='softmax')(head_model)
        #self.config['nbr_classes']
        self.head_model = head_model
                  
        # combine both models 
        self.model = keras.Model(self.base_model.input, head_model)


#         avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
#         output = keras.layers.Dense(20, activation='softmax')(avg)
#         model=keras.Model(inputs=base_model.input, outputs=output)
           
    def compile_model(self): 
        # optimizer
        if self.config['train_parameters']['optimizer'] == 'SGD':
            optimizer = tf.keras.optimizers.SGD(
                    learning_rate=self.config['train_parameters']['learning_rate'], momentum=0.9,
                    nesterov=False, name="SGD"
                )
        elif self.config['train_parameters']['optimizer'] == 'ADAM':
            optimizer = tf.keras.optimizers.Adam(lr=self.config['train_parameters']['learning_rate'])

        # metric
        metrics = [tf.keras.metrics.CategoricalAccuracy(),
                  tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top 3 categorical acccuracy'), 
                  tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top 5 categorical acccuracy')
                  ]
        
        # loss
        loss='categorical_crossentropy'
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
        
    def train(self, name_run, notes, tags):
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            try:
                # Currently, memory growth needs to be the same across GPUs
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.experimental.list_logical_devices('GPU')
                print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
            except RuntimeError as e:
                # Memory growth must be set before GPUs have been initialized
                print(e)
            
        # setup logging
        if self.config['logging_wandb']:
            # w&b 
            wandb.init(name=name_run, 
                   project=self.project_name,
                   notes=notes, 
                   tags=tags,
                   entity='cv-task-2')

            # save usefull config to w&b
            wandb.config.learning_rate = self.config['train_parameters']['learning_rate']
            wandb.config.batch_size = self.config['train_parameters']['batch_size']
            wandb.config.epochs = self.config['train_parameters']['epochs']
            wandb.config.steps_per_epoch = self.config['train_parameters']['steps_per_epoch']
             
        # build model 
        self.build()

        # set model parts trainable or not
        if self.config['train_base_model'] == False: 
            print('freezing base model layers')
            for layer in self.base_model.layers:
                layer.trainable = False
        if self.config['train_head_model'] == False: 
            print('freezing head model layers')
            for layer in self.head_model.layers:
                layer.trainable = False
        
        
        # compile model
        self.compile_model()
        
        if self.config['logging_wandb']:
            # set save_model true if you want wandb to upload weights once run has finished (takes some time)
            clbcks = [WandbCallback(save_model=False)]
        else: 
            clbcks = []

        
        # start training 
        history=self.model.fit(
                    x = self.dataset.train_generator(batch_size=self.config['train_parameters']['batch_size']),
                    steps_per_epoch = self.config['train_parameters']['steps_per_epoch'],
                    epochs=self.config['train_parameters']['epochs'], 
                    validation_data=self.dataset.validation_generator(batch_size=self.config['train_parameters']['batch_size']),
                    validation_steps=20, 
                    callbacks=clbcks
        )
        
        #workers=multiprocessing.cpu_count(),
        #use_multiprocessing=True,
    
    def prepare_for_inference(self, model_weights_path): 
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            try:
                # Currently, memory growth needs to be the same across GPUs
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.experimental.list_logical_devices('GPU')
                print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
            except RuntimeError as e:
                # Memory growth must be set before GPUs have been initialized
                print(e)
        self.build()
        self.model.load_weights(model_weights_path)
    
    def show_heatmap_prediction(self, image_id):
        LAYER_NAME=self.heatmap_layer_name
        im = np.load('data/test/img/test_{}.npy'.format(image_id))
        pre_im = self.dataset.prepare_test_image(im)
        batch = np.expand_dims(pre_im, axis=0)
    
        pred = self.model.predict(batch)
        idx=np.argmax(pred)
        score = np.round(pred[0][idx]/np.sum(pred),4)
        label=self.dataset.label_names[idx]

        grad_model = tf.keras.models.Model([self.model.inputs], [self.model.get_layer(LAYER_NAME).output, self.model.output])

        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(batch)
            loss = predictions[:, idx]

        output = conv_outputs[0]
        grads = tape.gradient(loss, conv_outputs)[0]

        gate_f = tf.cast(output > 0, 'float32')
        gate_r = tf.cast(grads > 0, 'float32')
        guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

        weights = tf.reduce_mean(guided_grads, axis=(0, 1))

        cam = np.ones(output.shape[0: 2], dtype = np.float32)

        for i, w in enumerate(weights):
            cam += w * output[:, :, i]

        cam = cv2.resize(cam.numpy(), (224, 224))
        cam = np.maximum(cam, 0)
        heatmap = (cam - cam.min()) / (cam.max() - cam.min())

        cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
        og_im = cv2.cvtColor(im.astype('uint8'), cv2.COLOR_RGB2BGR)

        og_im = cv2.resize(og_im, (224, 224))


        output_image = cv2.addWeighted(og_im, 0.7, cam, 1, 0)


        fig, axes = plt.subplots(1,2, figsize=(30,15))
        axes[1].imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
        axes[0].imshow(im)
        axes[0].set_title('prediction: {}, score: {}'.format(label, np.round(100*score,2)), fontsize=25)
        plt.show()

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
class XceptionModel(ClassifactionModel): 
    def __init__(self, config):
        # setup model name for wandb
        self.project_name = 'Xception'
        self.heatmap_layer_name='block14_sepconv2_act'
        # define the base model
        self.base_model = keras.applications.xception.Xception(weights="imagenet",
                                                               include_top=False)
        # super takes care of the rest
        super().__init__(config)
        
        # feed preprocessor function 
        self.dataset.feed_preprocess_function(keras.applications.xception.preprocess_input)

In [ ]:
config = {
    'name': 'XceptionModel',
    'logging_wandb': False,  #nice tool for tracking a run. make and account on wandb.ai and I will add you to this project
    'weights': "imagenet", # 'imagenet', #None, 
    'nbr_classes': 20,
    'input_shape': (224, 224, 3),
    'train_base_model': True, # whether to train the head and or base model
    'train_head_model': True, 
    'train_parameters': {
        'optimizer': 'ADAM',
        'epochs': 5,
        'batch_size': 64,
        'learning_rate': 0.00001, 
        'steps_per_epoch': 2000
    },
    'dataset': {
        'train_fraction': 0.9,
        'input_shape': (224, 224),
        'augmentation': True, # whether to augment images or not
        'uniform_sample_probabilities': False
    },
    'head_model': {
        'head_model_units': [], 
        'add_dropout':      [],
        'activation': 'relu'
    }
}

In [ ]:
Xception = XceptionModel(config)
Xception.prepare_for_inference('weights/Xception_finetuned.h5')

In [ ]:
Xception.show_heatmap_prediction(60)

In [ ]:
class ResNet50Model(ClassifactionModel): 
    def __init__(self, config):
        # setup model name for wandb
        self.project_name = 'resnet50'
        self.heatmap_layer_name='conv5_block3_out'
        # define the base model
        self.base_model = keras.applications.ResNet50V2(weights="imagenet",
                                                               include_top=False)
        # super takes care of the rest
        super().__init__(config)
        
        # feed preprocessor function 
        self.dataset.feed_preprocess_function(keras.applications.resnet_v2.preprocess_input)
config = {
    'name': 'ResNet50',
    'logging_wandb': False,  #nice tool for tracking a run. make and account on wandb.ai and I will add you to this project
    'weights': "imagenet", # 'imagenet', #None, 
    'nbr_classes': 20,
    'input_shape': (224, 224, 3),
    'train_base_model': False, # whether to train the head and or base model
    'train_head_model': True, 
    'train_parameters': {
        'optimizer': 'ADAM',
        'epochs': 3,
        'batch_size': 64,
        'learning_rate': 0.000001, 
        'steps_per_epoch': 2000
    },
    'dataset': {
        'train_fraction': 0.9,
        'input_shape': (224, 224),
        'augmentation': True, # whether to augment images or not
        'uniform_sample_probabilities': False
    },
    'head_model': {
        'head_model_units': [1024, 1024], 
        'add_dropout':      [False, False],
        'activation': 'relu'
    }
}

In [ ]:
name_run='ResNet50_firstTry'
notes='First try for finetuning resnet50 on pretrained imagenet weights. Data augmentation turned on. '
tags = ['resnet50', 'head = []', 'head = []', 'Augmentation applied', 'uniform class distribution']
resnet50=ResNet50Model(config)
resnet50.train(name_run, notes, tags)

In [ ]:
resnet50=ResNet50Model(config)
resnet50.prepare_for_inference('weights/resnet50.h5')

In [ ]:
idx=109
resnet50.show_heatmap_prediction(idx)

In [ ]:
resnet50.model.get_layer('conv5_block3_out').output

In [ ]:
#resnet50.model.summary()
resnet50.model.save_weights('resnet50_noOverfitting.h5')

# segmentation

In [ ]:
class Segmentationodel(ClassifactionModel): 
    """
        Main class implementing all functions necessary to train and/or use a classification model 
        This class has to be overwritten for each specific model of interest, where the base model should be implemented.
    """
    def __init__(self, config): 
        self.config = config 

        # initialize dataset
        self.dataset = Dataset_Segmentation(config['dataset'])
              

    def set_config(self, config):
        self.config = config 
        self.dataset = Dataset_Segmentation(config['dataset'])
    
    def show_heatmap_prediction(self, image_id):
        print('No')
        
    def predict(self, X, show=False):
        og_size = X.shape
        resized, _ = self.dataset.prepare_test_image(X, None)
        batch = np.expand_dims(resized, axis=0)
        y = self.model.predict(batch)
        mask = np.argmax(y, -1)[0].reshape(224,224)
        #stacked_img = np.array(np.stack((mask,)*3, axis=-1), dtype=np.float32)
        #print(stacked_img.shape)
        og_mask = cv2.resize(mask.astype('float32'), (og_size[1], og_size[0]))
                             
        class_names = get_VOC2012_classes()
        class_ids = np.unique(mask.reshape(-1))
                             
        if show:
            fig, axes = plt.subplots(figsize=(20,10))
            axes.imshow(X)
            axes.imshow(og_mask, alpha=0.5)
            for idx in class_ids:
                px_idx = np.where(og_mask == idx)
                px=np.random.choice(px_idx[0])
                py=np.random.choice(px_idx[1])
                axes.text(px,py, '{}'.format(class_names[idx]), fontsize=15)
            fig.show()
        return mask

# deeplab

In [ ]:
class deepLabModel(Segmentationodel):
    def __init__(self, config): 
        self.project_name = 'deepLab'
        super().__init__(config)
    
    def build(self):
        model = Deeplabv3(weights=None, input_tensor=None, infer=False,
                          input_shape=self.config['input_shape'], classes=21,
                          backbone=self.config['backbone'], OS=16, alpha=1)
        
        base_model = keras.Model(model.input, model.layers[-5].output)
        self.base_model = base_model
        
        if self.config['backbone']=='xception':
            scale = 4
        else:
            scale = 8
        sz=self.config['input_shape']
        x = tf.keras.layers.Conv2D(21, (1, 1), padding='same', name='conv_upsample')(base_model.output)
        x = tf.keras.layers.Lambda(lambda x: tf.compat.v1.image.resize_bilinear(x,size=(sz[0],sz[1])))(x)
        x = tf.keras.layers.Reshape((sz[0]*sz[1], -1)) (x)
        x = tf.keras.layers.Activation('softmax', name = 'pred_mask')(x)
        model = keras.Model(base_model.input, x, name='deeplabv3p')

        self.model = model
        return model
    
    def compile(self):
        # optimizer
        if self.config['train_parameters']['optimizer'] == 'SGD':
            optimizer = tf.keras.optimizers.SGD(
                    learning_rate=self.config['train_parameters']['learning_rate'], momentum=0.9,
                    nesterov=False, name="SGD"
                )
        elif self.config['train_parameters']['optimizer'] == 'ADAM':
            optimizer = tf.keras.optimizers.Adam(lr=self.config['train_parameters']['learning_rate'], 
                                                 epsilon=1e-8, decay=1e-6)

        # metric
        metrics = {'pred_mask' : [Jaccard, sparse_accuracy_ignoring_last_label]}
        
        # loss
        loss = sparse_crossentropy_ignoring_last_label
        
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
config = {
    'name': 'deepLab',
    'logging_wandb': False,  #nice tool for tracking a run. make and account on wandb.ai and I will add you to this project
    'nbr_classes': 20,
    'input_shape': (224, 224, 3),
    'train_base_model': True, # whether to train the head and or base model
    'train_head_model': True,
    'backbone': 'mobilenetv2', #xception
    'train_parameters': {
        'optimizer': 'ADAM',
        'epochs': 3,
        'batch_size': 64,
        'learning_rate': 0.000001, 
        'steps_per_epoch': 2000
    },
    'dataset': {
        'train_fraction': 0.9,
        'input_shape': (224, 224),
        'augmentation': True, # whether to augment images or not
        'uniform_sample_probabilities': False
    }
} 

In [ ]:
from keras.utils import to_categorical
seg = deepLabModel(config)
i=0
for (X,y) in seg.dataset.train_generator(10): 
#     print(y.shape)
#     new_y=list()
#     for yy in y: 
#         new_y.append(to_categorical(yy, num_classes=21))
#     y = np.array(new_y)
    print(X.shape)
    print(y.shape)
    i+=1
    if i > 3: 
        break

In [ ]:
name_run='deepLab'
notes='deeplab for segmentation'
tags = ['deepLab', 'Augmentation applied', 'uniform class distribution']
seg = deepLabModel(config)
seg.train(name_run, notes, tags)

In [ ]:
seg.prepare_for_inference('weights/mobilenetv2_original.h5')

In [ ]:
image_id=26
im = np.load('data/test/img/test_{}.npy'.format(image_id))
m=seg.predict(im, show=True)

In [ ]:
seg.model.summary()

In [ ]:
import sys

In [ ]:
sys.path.append('deeplab/')
from deeplab.deeplabv3p import Deeplabv3
from deeplab.utils import SegModel, get_VOC2012_classes

In [ ]:
seg = SegModel('', image_size=(224,224))
model = seg.create_seg_model(net='original',n=21, load_weights=True)

In [ ]:
image_id=6
im = np.load('data/test/img/test_{}.npy'.format(image_id))
plt.imshow(im)

In [ ]:
ds = Dataset_Segmentation(config['dataset'])
resized, _ = ds.prepare_test_image(im, None)
batch = np.expand_dims(resized, axis=0)

y = model(batch)

In [ ]:
mask = np.argmax(y, -1)[0].reshape(224,224)
plt.imshow(mask)
class_names = get_VOC2012_classes()
class_ids = np.unique(mask.reshape(-1))
for idx in class_ids:
    px_idx = np.where(mask == idx)
    px=np.random.choice(px_idx[0])
    py=np.random.choice(px_idx[1])
    plt.text(px,py, '{}'.format(class_names[idx]), fontsize=15)